## Data analytics assignment w/ Pandas and Socrata API

<p>
Point of the assingment is to demonstrate a technique or process from our course and apply it to some publicly available data set. While not a requirement for the assignment, we will also use this opportunity to utilize an API to retrieve the data which we will then filter and format into a useable form. (*We definitely aren't doing this extra work so I can have a reference document for the future. No way, that would be silly.*)

In order to access data via the SODA API we import the Socrata module. The imported configs holds our app token so that it won't be publicly available when uploaded to github. You can learn how to generate your own app token [here](https://support.socrata.com/hc/en-us/articles/360015776014-API-Keys), for free. 
</p>

In [1]:
import configs as cfg
import pandas as pd
from sodapy import Socrata

To access our data, we need identifiers for our endpoint. Our domain is data.seattle.gov and the specific dataset we are pulling from, [Code Complaints and Violations](https://data.seattle.gov/Community/Code-Complaints-and-Violations/ez4a-iug7) is identified by ez4a-iug7.

Using our token we build our client to query

In [2]:

data_url ="data.seattle.gov"
data_set="ez4a-iug7"

app_token = cfg.atoken
client=Socrata(data_url,app_token)
#client.timeout = 60


Unfortunately, the dataset we are looking at is quite big and we can only query for 50,000 rows are less per query. First, we'll check for the size of our dataset and from there we will download in pieces using offset.

In [6]:
count = client.get(data_set, select="COUNT(*)")

In [7]:
start = 0
block = 5000
results = []
while start < int(count[0]['COUNT']):
    results.extend(client.get(data_set, offset=start, limit=block))
    start +=block
    
df = pd.DataFrame.from_records(results)

df.to_csv("complaintsdata.csv")


## The dataset

<p>The dataset we've just pulled is a database of complaints filed the Seattle Depatment of Construction & Inspections. The data is really interesting because it contains all complaints filed by phone, online, and via the more recent "Find it, Fix it" app. Complaints can be varied from vendors using land improperly to elevators not working in buildings or potholes needing to be fixed. The data isn't curated into precise categories, as the categories may be user selected and thus not the most trustworthy, but that is dependent on the vagueness of reports. 

Besides inaccuracy due to the variety of user input, there are a few more caveats to using the database
* The dataset holds records from as far back as 2003, but the "Find it, Fix it" app wasn't released until 2013, which can skew any statistical outcomes if we look at data over time. Unfortunately we don't have any user statistics or complaint demographics so there doesn't seem to be any nice way to normalize
* The category of complaint isn't required to be set when making your submission, meaning if we try to find records pertaining to vegetation, we may have to filter records not only by the "Weeds" category, but also by descriptions that contain vegetation related words.
* The dataset is indexed on each complaint, so we need to count the dataset if we want to analyze by individual dates.
</p>

### Goal

<p>
We have a massive dataset with lots of stuff we don't care about. For the purposes of this project, we want to transform the dataset into the following:

* only reports of weeds, vegetation, or moss
* only reports that occur in a single calendar year, Jan 1 2021 to Dec 31 2021
* per day complaint counts for each day to look at trends
</p>






Now, we have just downloaded a massive amount of data, to the tune of an ~87 mb .CSV file. So, lets focus on filtering some of that info down.

In [ ]:

results = client.get(data_set, limit=50000)

df = pd.DataFrame.from_records(results)

df.to_csv("my_complaints_data.csv")

In [32]:
print(df.columns.tolist())
print(len(df.index))

['recordnum', 'recordtype', 'recordtypemapped', 'recordtypedesc', 'description', 'opendate', 'inspectiondate', 'lastinspresult', 'statuscurrent', 'originaladdress1', 'originalcity', 'originalstate', 'originalzip', 'link', 'latitude', 'longitude', 'location1']
50000


In [33]:
df[["description"]].head()

,description
0,"Elevator gets stuck at least 3 times a week, f..."
1,Concerned about RV pumpout that is currently b...
2,The vacant red apple building and adjacent bui...
3,Trailor and camper parked in back yard. Peopl...
4,"Illegal use of property.""Hawkers"" or vendors (..."


In [40]:
df[df['description'].str.contains("slip", case=False, na=False)]

,recordnum,recordtype,recordtypemapped,recordtypedesc,description,opendate,inspectiondate,lastinspresult,statuscurrent,originaladdress1,originalcity,originalstate,originalzip,link,latitude,longitude,location1
346,001347-03CP,Complaint,Request,Weeds,MOSS ON SIDEWALK SEVERAL ELDERLY PEOPLE HAVE S...,2003-12-18,"{'type': 'Point', 'coordinates': [-122.3099595...",Completed,Completed,2123 18TH AVE S,SEATTLE,WA,98144,{'url': 'https://cosaccela.seattle.gov/portal/...,47.58384244,-122.30995956,"{'latitude': '47.58384244', 'longitude': '-122..."
951,001303-04CP,Complaint,Request,Construction,"PUTTING ""Y"" CONNECTOR INTO 50 AMP ELECTRICAL S...",2004-02-02,"{'type': 'Point', 'coordinates': [-122.4058006...",Completed,Completed,7001 SEAVIEW AVE NW,SEATTLE,WA,98117,{'url': 'https://cosaccela.seattle.gov/portal/...,47.68002469,-122.40580069,"{'latitude': '47.68002469', 'longitude': '-122..."
1234,001586-04CP,Complaint,Request,Vacant Building,"Robert VanBogart, Bldg owner at 1915 2nd Ave, ...",2004-02-25,"{'type': 'Point', 'coordinates': [-122.3407681...",Completed,Completed,116 STEWART ST,SEATTLE,WA,98101,{'url': 'https://cosaccela.seattle.gov/portal/...,47.61094340,-122.34076814,"{'latitude': '47.6109434', 'longitude': '-122...."
1897,002249-04CP,Complaint,Request,NaN,Sewage or mudd coming up from the drain in the...,2004-04-21,"{'type': 'Point', 'coordinates': [-122.3549439...",NaN,Completed,13410 GREENWOOD AVE N,SEATTLE,WA,98133,{'url': 'https://cosaccela.seattle.gov/portal/...,47.72678668,-122.35494399,"{'latitude': '47.72678668', 'longitude': '-122..."
1907,002259-04CP,Complaint,Request,Weeds,Vegetation (including moss) encroaching onto s...,2004-04-21,"{'type': 'Point', 'coordinates': [-122.3660710...",Completed,Completed,1425 WILLARD AVE W,SEATTLE,WA,98119,{'url': 'https://cosaccela.seattle.gov/portal/...,47.63209430,-122.36607106,"{'latitude': '47.6320943', 'longitude': '-122...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48194,1020888-VI,Notice of Violation,Case,NaN,Exterior uncovered stairs to this apartment ge...,2009-10-16,"{'type': 'Point', 'coordinates': [-122.3146064...",Compliance Achieved,Completed,1213 NE 52ND ST,SEATTLE,WA,98105,{'url': 'https://cosaccela.seattle.gov/portal/...,47.66631276,-122.31460646,"{'latitude': '47.66631276', 'longitude': '-122..."
48485,1021184-VI,Notice of Violation,Case,Land Use,"At this general location, there are eight RV's...",2009-11-20,"{'type': 'Point', 'coordinates': [-122.3680024...",Compliance Achieved,Completed,1020 W EWING ST,SEATTLE,WA,NaN,{'url': 'https://cosaccela.seattle.gov/portal/...,47.65358617,-122.36800240,"{'latitude': '47.65358617', 'longitude': '-122..."
48850,001079-10CP,Complaint,Request,Weeds,"Per online complaint ""House is located between...",2010-01-11,"{'type': 'Point', 'coordinates': [-122.3847347...",Completed,Completed,8432 42ND AVE SW,SEATTLE,WA,98136,{'url': 'https://cosaccela.seattle.gov/portal/...,47.52792291,-122.38473471,"{'latitude': '47.52792291', 'longitude': '-122..."
49013,001242-10CP,Complaint,Request,Weeds,"Shrubs encroaching onto street, creating hazar...",2010-01-28,"{'type': 'Point', 'coordinates': [-122.3217829...",Completed,Completed,2600 HARVARD AVE E,SEATTLE,WA,98102,{'url': 'https://cosaccela.seattle.gov/portal/...,47.64338725,-122.32178295,"{'latitude': '47.64338725', 'longitude': '-122..."


In [68]:
metadata=client.get_metadata(data_set)
[x['name'] for x in metadata['columns']]

['RecordNum',
 'RecordType',
 'RecordTypeMapped',
 'RecordTypeDesc',
 'Description',
 'OpenDate',
 'LastInspDate',
 'LastInspResult',
 'StatusCurrent',
 'OriginalAddress1',
 'OriginalCity',
 'OriginalState',
 'OriginalZip',
 'Link',
 'Latitude',
 'Longitude',
 'Location1']

In [72]:
rawweeds = client.get(data_set, where= "recordtypedesc = 'Weeds'", limit=50000)
weeds = pd.DataFrame.from_records(results)


In [73]:
weeds.shape

(50000, 17)